In [ ]:
# 1. Google 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 2. 필요한 라이브러리 설치
!pip install pydub

In [ ]:
# 3. 라이브러리 임포트
import os
import random
from pydub import AudioSegment
from openpyxl import Workbook

In [ ]:
def mix_audio_files(audio_files, output_path, silence_thresh=-50.0, target_length_ms=5000):
    """
    여러 오디오 파일을 믹싱하여 하나의 오디오 파일로 출력합니다.
    """
    first_audio = AudioSegment.from_file(audio_files[0])
    second_audio = AudioSegment.from_file(audio_files[1])
    mixed_audio = first_audio.overlay(second_audio)

    final_audio = mixed_audio[:target_length_ms]
    final_audio.export(output_path, format="wav")
    #print(f"{os.path.basename(output_path)} saved: mix {first_audio} and {second_audio}")

In [ ]:
import os
import random
from openpyxl import Workbook
import shutil

def create_mixed_audio_files(animal_base_path, human_base_path, output_folder, num_files, spreadsheet_path="file_paths.xlsx"):
    """
    animal 폴더와 human 폴더에서 랜덤으로 파일을 선택하여 믹싱하고,
    'train', 'test', 'valid' 폴더에 지정된 비율로 분류하여 저장합니다.
    """
    # 경로 설정
    # animal_base = os.path.join(animal_base_path, 'animal')
    # human_base = os.path.join(human_base_path, 'human')

    # 출력 폴더 및 하위 폴더 생성
    subsets = ['train', 'test', 'valid']
    subset_counts = {'train': 400, 'test': 60, 'valid': 140}

    for subset in subsets:
        # os.makedirs(os.path.join(output_folder, subset), exist_ok=True)
        # os.makedirs(f'/content/drive/MyDrive/Dataset/Human_Animal/s1/{subset}', exist_ok=True)
        # os.makedirs(f'/content/drive/MyDrive/Dataset/Human_Animal/s2/{subset}', exist_ok=True)

        os.makedirs(os.path.join(output_folder, subset, 's1'), exist_ok=True)
        os.makedirs(os.path.join(output_folder, subset, 's2'), exist_ok=True)
        os.makedirs(os.path.join(output_folder, subset, 'mixed_data'), exist_ok=True)

    # 스프레드시트 초기화
    wb = Workbook()
    ws = wb.active
    ws.append(["index", "animal-file", "human-file", "subset"])  # 첫 번째 row 설정

    # 파일 생성 및 저장
    file_index = {'train': 0, 'test': 0, 'valid': 0}  # 각 subset의 생성된 파일 수
    for i in range(num_files):
        # 랜덤으로 animal 폴더와 파일 선택
        animal_folders = [f for f in os.listdir(animal_base_path) if os.path.isdir(os.path.join(animal_base_path, f))]
        random_animal_folder = random.choice(animal_folders)
        animal_folder_path = os.path.join(animal_base_path, random_animal_folder)
        animal_files = [f for f in os.listdir(animal_folder_path) if os.path.isfile(os.path.join(animal_folder_path, f))]
        selected_animal_file = random.choice(animal_files)
        selected_animal_path = os.path.join(animal_folder_path, selected_animal_file)

        # 랜덤으로 human 파일 선택
        human_files = [f for f in os.listdir(human_base_path) if os.path.isfile(os.path.join(human_base_path, f))]
        selected_human_file = random.choice(human_files)
        selected_human_path = os.path.join(human_base_path, selected_human_file)

        # 선택된 파일 이름 출력
        print(f"Animal_Human_{i + 1}.wav saved: mix animal={selected_animal_file}, human={selected_human_file}")

        # 파일 믹싱 및 저장
        # 현재 subset 결정
        for subset, count in subset_counts.items():
            if file_index[subset] < count:
                current_subset = subset
                break

        # Mixed 파일 저장
        # subset_folder = os.path.join(output_folder, current_subset, 'mixed_data')
        # output_path = os.path.join(subset_folder, f"Animal_Human_{i + 1}.wav")
        # mix_audio_files([selected_animal_path, selected_human_path], output_path=output_path)
        mixed_folder = os.path.join(output_folder, current_subset, 'mixed_data')
        mixed_output_path = os.path.join(mixed_folder, f"Animal_Human_{i + 1}.wav")
        mix_audio_files([selected_animal_path, selected_human_path], output_path=mixed_output_path)

        # 원본 파일 복사 및 저장
        # s1_path = f'/content/drive/MyDrive/Dataset/Human_Animal/s1/{current_subset}/Animal_Human_{i + 1}.wav'
        # s2_path = f'/content/drive/MyDrive/Dataset/Human_Animal/s2/{current_subset}/Animal_Human_{i + 1}.wav'
        s1_folder = os.path.join(output_folder, current_subset, 's1')
        s2_folder = os.path.join(output_folder, current_subset, 's2')
        s1_path = os.path.join(s1_folder, f"Animal_Human_{i + 1}.wav")
        s2_path = os.path.join(s2_folder, f"Animal_Human_{i + 1}.wav")
        shutil.copy(selected_animal_path, s1_path)
        shutil.copy(selected_human_path, s2_path)

        # 한 줄씩 스프레드시트에 기록 (인덱스와 파일 경로, subset 정보)
        ws.append([i + 1, f"{random_animal_folder}/{selected_animal_file}",
                   selected_human_file, current_subset])

        # 해당 subset의 파일 개수 증가
        file_index[current_subset] += 1

    # 엑셀 저장
    wb.save(spreadsheet_path)
    print(f"Spreadsheet saved at {spreadsheet_path}")
    return wb

In [ ]:
animal_base_path = '/content/drive/MyDrive/Dataset/RawData/animal'  # 기본 경로
human_base_path = '/content/drive/MyDrive/Dataset/RawData/human'

output_folder = '/content/drive/MyDrive/Dataset/mk_1800_dataset/'  # 출력 폴더
num_files = 600  # 생성할 믹싱 파일 수 #16000

wb = create_mixed_audio_files(animal_base_path, human_base_path, output_folder, num_files=num_files)

Animal_Human_1.wav saved: mix animal=2.까마귀_177.wav, human=430o0b06.wav_normalized.wav
Animal_Human_2.wav saved: mix animal=Bird_21.wav, human=440s0607.wav_normalized.wav
Animal_Human_3.wav saved: mix animal=trimmed_S-211026_A_103_D_441_0001.mp3의 사본, human=020o030y.wav_normalized.wav
Animal_Human_4.wav saved: mix animal=trimmed_S-210808_A_204_D_013_0001.mp3의 사본, human=002o0f0p.wav_normalized.wav
Animal_Human_5.wav saved: mix animal=trimmed_S-210930_A_102_D_014_0001.mp3의 사본, human=MMJB1_normalized.wav
Animal_Human_6.wav saved: mix animal=trimmed_S-211026_A_103_D_044_0001.mp3의 사본, human=408o030n.wav_normalized.wav
Animal_Human_7.wav saved: mix animal=1.고라니_51.wav, human=FSGF0_normalized.wav
Animal_Human_8.wav saved: mix animal=Bear_1.wav, human=40bo031a.wav_normalized.wav
Animal_Human_9.wav saved: mix animal=trimmed_S-211114_A_105_L_006_0001.mp3의 사본, human=001o0t0d.wav_normalized.wav
Animal_Human_10.wav saved: mix animal=Bird_13.wav, human=p265_normalized.wav
Ani